# Init & Login

In [1]:
from math import ceil
from random import randrange
from time import sleep

import spotipy
import spotipy.util as util
from IPython.display import clear_output

## Client Info ##
CLIENT_ID     = ""
CLIENT_SECRET = ""
CLIENT_SCOPE  = "user-follow-modify playlist-modify-private playlist-modify-public"
USER_NAME     = "31ytgsr7wdmiaroy77msqpiupdsi"
REDIR_URI     = "https://github.com/jwatson-CO-edu/yt_shuffle_so_good"
AUTH_URL      = 'https://accounts.spotify.com/api/token'
BASE_URL      = 'https://api.spotify.com/v1/'
## API Info ##
_RESPONSE_LIMIT = 100

with open( "../keys/spot_ID.txt" , 'r' ) as f:
    CLIENT_ID = f.readlines()[0].strip()

with open( "../keys/spot_SECRET.txt" , 'r' ) as f:
    CLIENT_SECRET = f.readlines()[0].strip()

token = None
token = util.prompt_for_user_token(
    username      = USER_NAME,
    scope         = CLIENT_SCOPE,
    client_id     = CLIENT_ID,
    client_secret = CLIENT_SECRET,
    redirect_uri  = REDIR_URI
)

print( token )

spot = spotipy.Spotify( auth = token )
clear_output( wait = True )
sleep( 2 )
print( "TOKEN OBTAINED" )

TOKEN OBTAINED


# Playlist Functions

In [2]:

def get_playlist_length( playlist_ID ):
    """ Get the number of total tracks in the playlist """
    response = spot.user_playlist_tracks(
        CLIENT_ID, 
        playlist_ID, 
        fields = 'items,uri,name,id,total', 
        limit  = _RESPONSE_LIMIT
    )
    return response['total']


def reorder_playlist_by_chunks( playlist_ID, chunkSize, Nmoves = None, verbose = False ):
    """ Shuffle the playlist a chunk at a time """
    Ntracks = get_playlist_length( playlist_ID )
    Nchunks = int( ceil( Ntracks / chunkSize ) )
    if Nmoves is None:
        Nmoves = int( Ntracks / chunkSize )
    for i in range( Nmoves ):
        bgnDex   = randrange(0,Nchunks) * chunkSize
        moveSize = min( chunkSize, Ntracks - bgnDex - 1 )
        sendDex  = randrange(0,Ntracks-1)
        try:
            response = spot.playlist_reorder_items(
                playlist_ID, 
                bgnDex, 
                sendDex, 
                range_length = chunkSize
            )
            if verbose:
                print( f"Iteration {i+1}, {bgnDex}-{bgnDex+moveSize} --> {sendDex}:\n\t{response}" )
        except spotipy.SpotifyException as ex:
            if verbose:
                print( f"ERROR, {bgnDex}-{bgnDex+moveSize} --> {sendDex}:\n\t{ex}" )


def complete_reorder_entire_playlist( playlist_ID, verbose = False, phasePause_s = 5 ):
    """ Completely shuffle the playlist in decreasing chunks """
    Ntracks = get_playlist_length( playlist_ID )
    ## Phase 1: Chunks of 100 ##
    if verbose:
        print("\nPhase 1: Chunks of 100\n")
    reorder_playlist_by_chunks( playlist_ID, 100, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 2: Chunks of 10 ##
    if verbose:
        print("\nPhase 2: Chunks of 10\n")
    reorder_playlist_by_chunks( playlist_ID, 10, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 3: Chunks of 5, 1/2 ##
    if verbose:
        print("\nPhase 3: Chunks of 5, 1/2\n")
    Nmoves = int( ceil( Ntracks / 10 ) )
    reorder_playlist_by_chunks( playlist_ID, 5, Nmoves, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 3: 1 at a Time, 1/3 ##
    if verbose:
        print("\nPhase 3: 1 at a Time, 1/3\n")
    Nmoves = int( ceil( Ntracks / 3 ) )
    reorder_playlist_by_chunks( playlist_ID, 1, Nmoves, verbose = verbose )

# Shuffle Playlists!

In [3]:
playlist = {
    'study01' : "0a2qoe6S7lYeZ6nlhZdA0v",
    'study02' : "6gbtR2cBq5PvkghidCvvGk",
    'study03' : "3o3lN2qntdEV7UKTuuC77K",
    'study04' : "41sFSisljvBDMBXtpp5NIw",
    'study05' : "02iS5AFGp8YVuUUqcQf8ys",
    'study06' : "6KI7A4MWrSM7EyKRUjxIi1",
    'study07' : "3V055Md2JdrUT8tX0af7di",
    'study08' : "0tspdJlwSgiyf2O9PO6QaP",
    'study09' : "5mHRBFoQtYy2izeZ66pG95",
    'study10' : "3832xeKGEOAXFJqE4K8kIq",
    'study11' : "65MXR4dubPL9t0P4dgTWvn",
    'study12' : "0ecSAfnD4CulIVnLt26ukI",
    'study13' : "7K9ucByFRgDuZk8KMHeJkL",
    'cringe'  : "2AAUYlKM1nXKHkZUpUSFbv",
}

complete_reorder_entire_playlist( playlist['cringe'], verbose = True, phasePause_s = 5 )


Phase 1: Chunks of 100

Iteration 1, 200-300 --> 283:
	{'snapshot_id': 'AAAGvDCQv8nRmSWVA6bRtqxrf6iPokv3'}
Iteration 2, 200-300 --> 178:
	{'snapshot_id': 'AAAGvee9x+GSs8R6/EVfg+/givWBWos1'}


HTTP Error for PUT to https://api.spotify.com/v1/playlists/2AAUYlKM1nXKHkZUpUSFbv/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


Iteration 3, 100-200 --> 269:
	{'snapshot_id': 'AAAGvpRhhDbsVsjAXLDpfkxLx4f0/8t/'}
ERROR, 400-438 --> 289:
	http status: 400, code:-1 - https://api.spotify.com/v1/playlists/2AAUYlKM1nXKHkZUpUSFbv/tracks:
 Tracks selected to be reordered are out of bounds, reason: None

Phase 2: Chunks of 10

Iteration 1, 320-330 --> 236:
	{'snapshot_id': 'AAAGvxSWkwKsSJ4rwNNLA05ePCpPy8Ng'}
Iteration 2, 150-160 --> 320:
	{'snapshot_id': 'AAAGwNSATwROiyVaDgGxIFYITO8i20K8'}
Iteration 3, 70-80 --> 279:
	{'snapshot_id': 'AAAGwdoqvahXadCXke4UQNKJXSM93q4u'}
Iteration 4, 280-290 --> 220:
	{'snapshot_id': 'AAAGwsPwghN0S2ur06r9D/g4MYhZ1Gpo'}
Iteration 5, 320-330 --> 380:
	{'snapshot_id': 'AAAGw0Aylt9eFEBYKBMomQSn2QeoPJW1'}
Iteration 6, 40-50 --> 392:
	{'snapshot_id': 'AAAGxNmL2KgpZmkngDqAuknpMGdeBuRw'}
Iteration 7, 190-200 --> 421:
	{'snapshot_id': 'AAAGxanJAqoJ6l+ZIlaFbhg32YtGlZ8N'}
Iteration 8, 160-170 --> 325:
	{'snapshot_id': 'AAAGxuq1zYk6/01rwit1Aau5QSD4ynR3'}
Iteration 9, 410-420 --> 382:
	{'snapshot_id': 

HTTP Error for PUT to https://api.spotify.com/v1/playlists/2AAUYlKM1nXKHkZUpUSFbv/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


Iteration 11, 165-170 --> 262:
	{'snapshot_id': 'AAAG9FDa8C0h9cIEBRZ15FmSOvGaOmlE'}
ERROR, 435-438 --> 406:
	http status: 400, code:-1 - https://api.spotify.com/v1/playlists/2AAUYlKM1nXKHkZUpUSFbv/tracks:
 Tracks selected to be reordered are out of bounds, reason: None
Iteration 13, 30-35 --> 135:
	{'snapshot_id': 'AAAG9YoLMvvl1+IptSSUcz2OWNv9zd2h'}
Iteration 14, 375-380 --> 55:
	{'snapshot_id': 'AAAG9pCtVnz2mBd0TTaGj+yxfjeA5wju'}
Iteration 15, 155-160 --> 402:
	{'snapshot_id': 'AAAG98CE//eHGUkax3ePSlSrDeGLDEYj'}
Iteration 16, 250-255 --> 278:
	{'snapshot_id': 'AAAG+Pvxh6R0KHixyuFi6s/fhnUiD4Ln'}
Iteration 17, 210-215 --> 340:
	{'snapshot_id': 'AAAG+cl8XwxLuv3lp2ctRzWWwadQRzfG'}
Iteration 18, 285-290 --> 235:
	{'snapshot_id': 'AAAG+qT076dnSlhHqKLBk8szV/kLzvAw'}
Iteration 19, 350-355 --> 285:
	{'snapshot_id': 'AAAG+3GGjq3IIqttLLXZnCzMYh5HbY4x'}
Iteration 20, 385-390 --> 369:
	{'snapshot_id': 'AAAG/BXNaCf3e7/dv5ZVgTOrOEGO7/SO'}
Iteration 21, 405-410 --> 218:
	{'snapshot_id': 'AAAG/fqFHFPE